In [ ]:
# Focal Loss에 alpha 값을 클래스별로 설정 => 임의적인 숫자를 넣지 말고 평가 지표 결과로 했었어야 했다. 
self.vfl = VarifocalLoss(alpha=0.75, class_gammas=torch.tensor([
    2.5,  # General trash (예측이 어려운 클래스)
    1.5,  # Paper
    1.5,  # Paper pack
    1.0,  # Metal
    1.0,  # Glass
    1.5,  # Plastic
    1.5,  # Styrofoam
    1.5,  # Plastic bag
    2.5,  # Battery (예측이 어려운 클래스)
    2.5   # Clothing (예측이 어려운 클래스)
], device='cuda')) if use_vfl else None  # gamma는 Focal Loss의 난이도 보정 파라미터	

class VarifocalLoss(nn.Module):
    """
    Varifocal loss by Zhang et al.

    https://arxiv.org/abs/2008.13367.
    """

    def __init__(self, class_gammas=None, alpha=0.75):
        """Initialize the VarifocalLoss class with class-specific gamma values."""
        super().__init__()
        self.class_gammas = class_gammas  # 클래스별 gamma 값
        self.alpha = alpha  # alpha 값

    def forward(self, pred_score, gt_score, label):
        """Computes Varifocal Loss."""
        
        if self.class_gammas is None:
            gamma = 2.0  # 기본 gamma 값
        else:
            # 각 클래스에 해당하는 gamma 값을 가져옵니다. [1, 1, num_classes]
            gamma = self.class_gammas.unsqueeze(0).unsqueeze(0)
        
        # 가중치 계산
        weight = self.alpha * pred_score.sigmoid().pow(gamma) * (1 - label) + gt_score * label
        
        with autocast(enabled=False):
            # Varifocal Loss 계산
            loss = (
                F.binary_cross_entropy_with_logits(pred_score.float(), gt_score.float(), reduction="none") * weight
            ).mean(1).sum()
        
        return loss